# Stanford DCI 2023 DCI fellow summary

### Load packages

In [26]:
%reload_ext autoreload
%autoreload 2

import sys
import pprint
import pandas as pd
from IPython.display import HTML

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [27]:
!{sys.executable} -m pip install bs4

In [44]:
from uniflow import Context, PromptTemplate
from uniflow.flow.client import ExtractClient
from uniflow.flow.config import ExtractHTMLConfig
from uniflow.flow.client import TransformClient
from uniflow.flow.config  import TransformConfig
from uniflow.flow.config import PipelineConfig
from uniflow.op.model.model_config  import OpenAIModelConfig
from uniflow.pipeline import MultiFlowsPipeline

from uniflow.viz import Viz
from uniflow.flow.flow_factory import FlowFactory

FlowFactory.list()

{'extract': ['ExtractHTMLFlow',
  'ExtractImageFlow',
  'ExtractIpynbFlow',
  'ExtractMarkdownFlow',
  'ExtractPDFFlow',
  'ExtractTxtFlow'],
 'transform': ['TransformAzureOpenAIFlow',
  'TransformCopyFlow',
  'TransformGoogleFlow',
  'TransformGoogleMultiModalModelFlow',
  'TransformHuggingFaceFlow',
  'TransformLMQGFlow',
  'TransformOpenAIFlow'],
 'rater': ['RaterFlow']}

### Prepare the input data

We can not only load local html files by `filename`, but also load online html files by providing the `url`.

In [45]:
input_data = [
    {"filename": "https://dci.stanford.edu/fellow/a-j-agarwal/"},
    {"filename": "https://dci.stanford.edu/fellow/nancy-andrews/"},
    {"filename": "https://dci.stanford.edu/fellow/max-bosel/"},
    {"filename": "https://dci.stanford.edu/fellow/ana-botero/"},
    {"filename": "https://dci.stanford.edu/fellow/molly-c-campbell/"},
    {"filename": "https://dci.stanford.edu/fellow/franklin-carone/"},
    {"filename": "https://dci.stanford.edu/fellow/david-ciulla/"},
    {"filename": "https://dci.stanford.edu/fellow/ranjana-clark/"},
    {"filename": "https://dci.stanford.edu/fellow/jim-cowie/"},
    {"filename": "https://dci.stanford.edu/fellow/jason-green/"},
    {"filename": "https://dci.stanford.edu/fellow/michele-grieshaber/"},
    {"filename": "https://dci.stanford.edu/fellow/melissa-hollatz/"},
    {"filename": "https://dci.stanford.edu/fellow/kim-jabal/"},
    {"filename": "https://dci.stanford.edu/fellow/joan-jeffri/"},
    {"filename": "https://dci.stanford.edu/fellow/robin-joy/"},
    {"filename": "https://dci.stanford.edu/fellow/john-kleinheinz/"},
    {"filename": "https://dci.stanford.edu/fellow/marsha-kleinheinz/"},
    {"filename": "https://dci.stanford.edu/fellow/catherine-martineau/"},
    {"filename": "https://dci.stanford.edu/fellow/william-mccarthy/"},
    {"filename": "https://dci.stanford.edu/fellow/denis-morozov/"},
    {"filename": "https://dci.stanford.edu/fellow/roswitha-mueller/"},
    {"filename": "https://dci.stanford.edu/fellow/maria-pacheco/"},
    {"filename": "https://dci.stanford.edu/fellow/madhu-rao/"},
    {"filename": "https://dci.stanford.edu/fellow/jose-revuelta/"},
    {"filename": "https://dci.stanford.edu/fellow/veronica-rogers/"},
    {"filename": "https://dci.stanford.edu/fellow/jim-rowe/"},
    {"filename": "https://dci.stanford.edu/fellow/avanish-sahai/"},
    {"filename": "https://dci.stanford.edu/fellow/faye-sahai/"},
    {"filename": "https://dci.stanford.edu/fellow/nirmala-sankaran/"},
    {"filename": "https://dci.stanford.edu/fellow/sonja-k-schoenwald/"},
    {"filename": "https://dci.stanford.edu/fellow/ramon-segismundo-dba/"},
    {"filename": "https://dci.stanford.edu/fellow/steven-seleznow/"},
    {"filename": "https://dci.stanford.edu/fellow/felice-silk/"},
    {"filename": "https://dci.stanford.edu/fellow/david-silk/"},
    {"filename": "https://dci.stanford.edu/fellow/priscila-costa-snel-correa/"},
    {"filename": "https://dci.stanford.edu/fellow/luciano-snel-correa/"},
    {"filename": "https://dci.stanford.edu/fellow/timothy-a-steinert/"},
    {"filename": "https://dci.stanford.edu/fellow/keller-strother/"},
    {"filename": "https://dci.stanford.edu/fellow/sunit-survase/"},
    {"filename": "https://dci.stanford.edu/fellow/susan-vobejda/"},
    {"filename": "https://dci.stanford.edu/fellow/joseph-weber/"},
]

### Extract the html file via ExtractClient with customized post extract fn to transform data into desired format

In [46]:
def post_extract_fn(data):
    def get_subset(strings, start, end):
        try:
            start_index = strings.index(start)
            end_index = strings.index(end)
            return strings[start_index+1:end_index]
        except ValueError:
            return []
        
    return {"text": ["\n".join(get_subset(data['text'], "Select Page", "2023 FELLOWS & PARTNERS"))]}

extract_config = ExtractHTMLConfig(
    post_extract_fn=post_extract_fn
)

### Transform to the desired format via TransformClient

In [47]:
transform_config = TransformConfig(
    flow_name="TransformOpenAIFlow",
    model_config=OpenAIModelConfig(
        response_format={"type": "json_object"},
        temperature=0),
    prompt_template=PromptTemplate(
            instruction="""
            Please provide a concise summary of the professional background and achievements of the individual featured on this webpage, including key roles, achievements with details numbers, and educational background. Also, identify the primary industry in which this person has made their career. Ensure the summary is brief and informative, capturing the essence of their professional journey and industry involvement.
            Follow the format of the examples below to include both name, summary, and industry in response.
            """,
            # few_shot_prompt=[
            #     Context(
            #         context="""
            #         A.J. Agarwal is a Senior Managing Director in Blackstone's Real Estate Group, where he launched  Blackstone's U.S. Core+ real estate investing business, with over $125 billion of equity under management today.  He joined Blackstone in 1992 and is a member of the firm's Real Estate Investment Committee.  Most recently, he served as President and Director of Blackstone Real Estate Income Trust, Blackstone's largest investment vehicle with over $65 billion of market capitalization and over $125 billion of real estate assets.  Prior to launching Blackstone's Core+ real estate investing business in 2014, A.J. was Co-Head of U.S. Acquisitions for Blackstone's opportunistic equity investing business and he oversaw more than $50 billion of real estate investments across all real estate classes.  
            #         A.J. graduated from Princeton University, where he studied Politics and graduated magna cum laude and Phi Beta Kappa, and received his MBA from Stanford University Graduate School of Business. He is a member of the Council on Foreign Relations  as well as the Stanford Alumni Real Estate Council.
            #         He is married to Roswitha Mueller-Agarwal and has two children, a daughter (Stanford '26) and son who lives in New York City.  A.J. is an avid tennis player and cyclist.""",
            #         name="""A.J. Agarwal""",
            #         summary="""A.J. Agarwal, a Senior Managing Director at Blackstone's Real Estate Group, is notable for launching and managing the firm's U.S. Core+ real estate investing business, overseeing over $125 billion of equity. His career includes key positions such as President and Director of Blackstone Real Estate Income Trust and Co-Head of U.S. Acquisitions. Agarwal, a Princeton University and Stanford Graduate School of Business alumnus, has significantly impacted the real estate investment sector.""",
            #         industry="""Real Estate Investment"""
            #     ),
            #     Context(
            #         context="""...""",
            #         name="""...""",
            #         summary="""...""",
            #         industry="""..."""
            #     ),
            # ],
        )
    )


In [48]:
p = MultiFlowsPipeline(PipelineConfig(
    extract_config=extract_config,
    transform_config=transform_config,
))

In [49]:
output = p.run(input_data)

100%|██████████| 1/1 [00:03<00:00,  3.07s/it]


In [59]:
name = []
summary = []
industry = []
url = []
for o, u in zip(output, input_data):
    name.append(o[0]['output'][0]['response'][0]['name'])
    summary.append(o[0]['output'][0]['response'][0]['summary'])
    industry.append(o[0]['output'][0]['response'][0]['industry'])
    url.append(u['filename'])


# Sample data
data = {
    'Name': name,
    'Summary': summary,
    'Industry': industry,
    'URL': url
}

# Adjusting display settings to avoid truncation
pd.set_option('display.max_rows', None)  # Adjust to display all rows
pd.set_option('display.max_columns', None)  # Adjust to display all columns
pd.set_option('display.width', None)  # Adjust to ensure each row uses optimal width
pd.set_option('display.max_colwidth', None)  # Adjust to display full content of each cell

# Creating a DataFrame
df = pd.DataFrame(data)
df['URL'] = df['URL'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')


# Displaying the DataFrame
display(HTML(df.to_html(escape=False)))


,Name,Summary,Industry,URL
0,A.J. Agarwal,"A.J. Agarwal is a Senior Managing Director in Blackstone's Real Estate Group, where he launched Blackstone's U.S. Core+ real estate investing business, with over $125 billion of equity under management today. He joined Blackstone in 1992 and is a member of the firm's Real Estate Investment Committee. Most recently, he served as President and Director of Blackstone Real Estate Income Trust, Blackstone's largest investment vehicle with over $65 billion of market capitalization and over $125 billion of real estate assets. Prior to launching Blackstone's Core+ real estate investing business in 2014, A.J. was Co-Head of U.S. Acquisitions for Blackstone's opportunistic equity investing business and he oversaw more than $50 billion of real estate investments across all real estate classes. A.J. graduated from Princeton University, where he studied Politics and graduated magna cum laude and Phi Beta Kappa, and received his MBA from Stanford University Graduate School of Business. He is a member of the Council on Foreign Relations as well as the Stanford Alumni Real Estate Council. He is married to Roswitha Mueller-Agarwal and has two children, a daughter (Stanford '26) and son who lives in New York City. A.J. is an avid tennis player and cyclist.",Real Estate Investment,https://dci.stanford.edu/fellow/a-j-agarwal/
1,Nancy O. Andrews,"Nancy O. Andrews is a prominent figure in the community development industry, with a career spanning 40 years. She was the President and Chief Executive Officer of the Low Income Investment Fund (LIIF), where she grew the organization's assets from $35 million to $1 billion. Nancy's work influenced federal policy and led to capital innovations, such as outcomes-based investments like Equity with a Twist. She has also served on numerous boards and committees of community development and environmental organizations, including Bank of America's National Community Advisory Council and the Federal Reserve Board's Consumer Advisory Council. Nancy's career also includes work in the Clinton administration Departments of Treasury and Housing and Urban Development, as well as serving as the Deputy Director of the Ford Foundation's social investment portfolio.",Community Development,https://dci.stanford.edu/fellow/nancy-andrews/
2,Max Bosel,"Max Bosel has had an extensive career in public safety in the San Francisco Bay Area, beginning as a high school volunteer and culminating with 31 years as a peace officer. He served as the police chief of Mountain View, CA, and has wide-ranging leadership experiences in public safety. In 2021, Max founded ""Code 4 Finances,"" providing financial wellness education and coaching tailored to first responders. He holds a BA in Management from St. Mary's College, an MPA from Notre Dame de Namur University, and a Graduate Certificate in Financial Life Planning from Golden Gate University. Max is a graduate of the Harvard Kennedy School of Government's State and Local Government Executive Program and the FBI's National Academy. He is actively involved in non-profit work and has been married since 1995, enjoying an active lifestyle and extensive travel.",Public Safety and Financial Education,https://dci.stanford.edu/fellow/max-bosel/
3,Ana Mercedes Botero,"Ana Mercedes Botero is a Colombian lawyer and diplomat with extensive experience in international scenarios, including a diplomatic career at the United Nations. She currently serves as the Director of Social Innovation at the Development Bank of Latin America-Corporacion Andina de Fomento (CAF). Ana has been involved in international negotiations and played a key role in creating a community-driven development department aimed at empowering vulnerable groups and promoting social inclusion. She holds two master's degrees from Columbia University and has completed leadership programs at prestigious institutions such as Harvard. Ana's professional journey has been focused on social innovation 

## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example/model#examples)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>